In [92]:
import os
import requests
import json
import pandas as pd
import hvplot.pandas
import sqlalchemy
from sqlalchemy import inspect
from dotenv import load_dotenv
import numpy as np

In [63]:
load_dotenv()

True

In [64]:
database_connection_string = os.getenv("DATABASE_URL")
database_schema = os.getenv("DATABASE_SCHEMA")

display(type(database_connection_string))
display(type(database_schema))

str

str

In [65]:
engine = sqlalchemy.create_engine(database_connection_string)
inspector = inspect(engine)
inspector.get_table_names()

['token_attribute',
 'network',
 'collection',
 'contract_map',
 'token',
 'whale',
 'api',
 'trade',
 'social_media',
 'data_analysis']

In [66]:
os_top_collection_index = """
SELECT t.contract_id,
       c.name,
       DATE_TRUNC('day', t.timestamp) as year_day_month,
       SUM(t.volume) as total_volume,
       SUM(t.num_trades) as total_num_trades,
       SUM(t.unique_buyers) as total_unique_buyers
FROM trade t
INNER JOIN collection c ON c.contract_id = t.contract_id
INNER JOIN network n ON n.network_id = c.network_id
WHERE n.network_id = 'ethereum' 
AND c.name NOT IN ('','New 0x495f947276749Ce646f68AC8c248420045cb7b5eLock', 'pieceofshit', 'Uniswap V3 Positions NFT-V1','More Loot',
                  'NFTfi Promissory Note','dementorstownwtf','ShitBeast','mcgoblintownwtf','LonelyPop','Pablos','For the Culture','Hype Pass')
GROUP BY t.contract_id, c.name, DATE_TRUNC('day', t.timestamp)
HAVING MAX(avg_price) > 0
ORDER BY DATE_TRUNC('day', t.timestamp)  ASC
"""

In [67]:
os_top_collection_index_df = pd.read_sql_query(os_top_collection_index, con=engine)

In [68]:
os_top_collection_index_df

,contract_id,name,year_day_month,total_volume,total_num_trades,total_unique_buyers
0,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-23,3,20,8
1,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-24,4,20,7
2,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-25,2,13,7
3,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-26,2,18,8
4,ethereum:b47e3cd837ddf8e4c57f05d70ab865de6e193bbb,CryptoPunks,2017-06-27,8,35,8
...,...,...,...,...,...,...
14760,ethereum:ed5af388653567af2f388e6224dc7c4b3241c544,Azuki,2022-08-06,27,3,3
14761,ethereum:c36cf0cfcb5d905b8b513860db0cfe63f6cf9f5c,October London,2022-08-06,23,41,23
14762,ethereum:8a90cab2b38dba80c64b7734e58ee1db38b8992e,Doodles,2022-08-06,8,1,1
14763,ethereum:d2f668a8461d6761115daf8aeb3cdf5f40c532c6,Karafuru,2022-08-06,0,1,1


In [69]:
os_top_collection_index_df = os_top_collection_index_df[['year_day_month', 'total_volume', 'total_num_trades', 'total_unique_buyers']]

In [70]:
os_top_collection_index_df.columns = ['date', 'volume', 'num_trades', 'unique_buyers']

In [71]:
os_top_collection_index_df

,date,volume,num_trades,unique_buyers
0,2017-06-23,3,20,8
1,2017-06-24,4,20,7
2,2017-06-25,2,13,7
3,2017-06-26,2,18,8
4,2017-06-27,8,35,8
...,...,...,...,...
14760,2022-08-06,27,3,3
14761,2022-08-06,23,41,23
14762,2022-08-06,8,1,1
14763,2022-08-06,0,1,1


In [72]:
os_top_collection_index_df = os_top_collection_index_df[os_top_collection_index_df['date'] > '2020-12-31']

In [73]:
os_top_collection_index_volume = os_top_collection_index_df.groupby('date')['volume'].sum()
os_top_collection_index_num_trades = os_top_collection_index_df.groupby('date')['num_trades'].sum()
os_top_collection_index_unique_buyers = os_top_collection_index_df.groupby('date')['unique_buyers'].sum()

In [74]:
os_top_collection_index_unique_buyers

date
2021-01-01     217
2021-01-02     142
2021-01-03     108
2021-01-04      75
2021-01-05     103
              ... 
2022-08-02     501
2022-08-03     479
2022-08-04    2654
2022-08-05    2843
2022-08-06     287
Name: unique_buyers, Length: 583, dtype: int64

In [75]:
overlay_1 = os_top_collection_index_volume.hvplot(
    height=700, 
    width=1000, 
    yformatter='%.0f', 
    ylabel = 'Volume in ETH', 
    xlabel = 'Date',
    title = 'Open Sea Volume for Top Collections January 2021 to August 2022',
    color = 'green',
    label = 'Volume in ETH'
)

# OPTION ONE

In [76]:
overlay_1

:Curve   [date]   (volume)

# OPTION 2

In [89]:
overlay_2 = os_top_collection_index_num_trades.hvplot(
    height=700, 
    width=1000, 
    yformatter='%.0f', 
    ylabel = 'Number of Trades', 
    xlabel = 'Date',
    title = 'Open Sea Number of Trades for Top Collections January 2021 to August 2022',
    color = 'blue',
    label = 'Number of Trades'
)

In [90]:
overlay_2

:Curve   [date]   (num_trades)

# OPTION 3

In [87]:
overlay_3 = os_top_collection_index_unique_buyers.hvplot(
    height=700, 
    width=1000, 
    yformatter='%.0f', 
    xlabel = 'Date',
    ylabel = 'Nmber of Unique Buyers',
    title = 'Open Sea Unique Buyers for Top Collections January 2021 to August 2022',
    color = 'red',
    label = 'Number of Unique Buyers',
)

In [88]:
overlay_3

:Curve   [date]   (unique_buyers)

# OPTION 4

In [30]:
overlay = overlay_1 * overlay_2 * overlay_3

In [91]:
overlay.opts(
    title = 'Open Sea Top Collections January 2021 to August 2022',
    show_legend = True,
    legend_position = 'top_left'
)

:Overlay
   .Curve.Volume        :Curve   [date]   (volume)
   .Curve.Num_trades    :Curve   [date]   (num_trades)
   .Curve.Unique_buyers :Curve   [date]   (unique_buyers)

# OPTION 5

In [54]:
overlay_2_and_3 = (overlay_2 * overlay_3)
overlay_2_and_3.opts(
    title = 'Open Sea Number of Trades and Unique Buyers January 2021 to August 2022',
    show_legend = True,
    legend_position = 'top_left'
)

:Overlay
   .Curve.Number_of_Trades        :Curve   [date]   (num_trades)
   .Curve.Number_of_Unique_Buyers :Curve   [date]   (unique_buyers)

In [56]:
side_by_side = overlay_2_and_3 + overlay_1

In [59]:
side_by_side

:Layout
   .Overlay.I           :Overlay
      .Curve.Number_of_Trades        :Curve   [date]   (num_trades)
      .Curve.Number_of_Unique_Buyers :Curve   [date]   (unique_buyers)
   .Curve.Volume_in_ETH :Curve   [date]   (volume)